# TUTORIAL PYTORCH
## Paralelismo
###### Fuente: [Documentación Oficial de Pytorch](https://pytorch.org/tutorials/)
###### Edición y traducción por Cristobal Donoso O. 
###### Agosto 2018

En este tutorial se verá como ocupar varias GPU's en PyTorch. Primero, pondremos el modelo en la una GPU
```
device = torch.device("cuda:0")
model.to(device)
```


De la misma manera podemos cargar los tensores en GPU. Note que ```my_tensor.to(device)``` retorna una nueva copia del tensor  en GPU, en vez de sobreescribir el anterior.
```
mytensor = my_tensor.to(device)
```

Es natural ejecutar el forward y backward sobre multiples GPU's. Sin embargo, PyTorch usara solo una GPU por defecto. Facilmente, tu puedes correr tus operaciones sobre multiples GPU'S haciendo que tu modelo corra en paralelo usando ```DataParallel```
```
model = nn.DataParallel(model)
```

### Ejemplo

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Parameters and DataLoaders
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (device)

cpu


### Generando datos de juguete

In [10]:
class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
                         batch_size=batch_size, shuffle=True)

### Modelo Simple

In [11]:
class Model(nn.Module):
    # Our model

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size", input.size(),
              "output size", output.size())

        return output

### Modelo y datos en paralelo

In [12]:
model = Model(input_size, output_size)

if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  model = nn.DataParallel(model)

model.to(device)

Model(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

### Ejecutar el Modelo

In [13]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
          "output_size", output.size())

	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])


```DataParallel``` divide tus datos automaticamente y envia los trabajos hacia multiples modelos que se encuentran en varias GPUs. Despues que todos los trabajos hayan terminado DataParallel recolecta y mezcla todos los resultados antes de desplegarlos